In [28]:
import numpy as np
import tensorflow as tf
import time
import os
import json

In [29]:
# Cargar el dataset desde el archivo .npy
path1 = '../DatasetVGG19/test.npy'
path2 = '../DatasetVGG19/train.npy'
path3 = '../DatasetVGG19/validation.npy'

In [30]:
data_train = np.load(path2)
data_test = np.load(path1)
data_val = np.load(path3)

In [31]:

def unpack(array):
    x = array[:,:-2]
    y = array[:,-2:]
    y = np.argmax(y, axis=1)
    return x,y

def model_score_tf(X,y,model,dataset):
    test_loss, test_accuracy = model.evaluate(X, y)
    print(f"Precisión del modelo {dataset}: {test_accuracy * 100:.2f}%")
    return test_accuracy

def writeJson(tiempo,accu,metod):
    nuevo_registro = {
        'Tipo Ejecucion' : "GPU",
        'Metodo' : metod,
        'Epochs':20,
        'tiempo_ejecucion': tiempo,
        'accuracy' : accu
    }
    nombre_archivo = '../GPU_tiempos.json'

    # Leer el contenido existente, si el archivo ya existe
    if os.path.exists(nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            try:
                registros = json.load(archivo)
            except json.JSONDecodeError:
                registros = []
    else:
        registros = []

    # Agregar el nuevo registro
    registros.append(nuevo_registro)

    # Guardar los registros actualizados en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
        json.dump(registros, archivo, indent=4)

# MLPClassifier

In [32]:
X_train,y_train = unpack(data_train)
X_test,y_test = unpack(data_train)
X_val,y_val = unpack(data_val)

# Convertir las etiquetas a formato one-hot para TensorFlow
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [33]:
def MLP_Classifier (X,y,X_test,y_test):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(500, activation='relu', input_shape=(1000,)),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X, y, epochs=20, validation_data=(X_test, y_test),verbose=0)
    return model

In [34]:
print("Entrenamiento MLP Classifier")

inicio = time.time()

MLPC_model = MLP_Classifier(X_train,y_train,X_test,y_test)

fin = time.time()
tiempo_ejecucion_MLPC = fin - inicio

accuracy = model_score_tf(X_test,y_test,MLPC_model,"Test")
model_score_tf(X_val,y_val,MLPC_model,"Val")

print(f"El tiempo de ejecucion fue de {tiempo_ejecucion_MLPC:.2f} segundos")

Entrenamiento MLP Classifier
475/475 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7398 - loss: 0.5486
Precisión del modelo Test: 74.36%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step - accuracy: 0.5000 - loss: 0.7514
Precisión del modelo Val: 50.00%
El tiempo de ejecucion fue de 37.93 segundos


In [35]:
writeJson(tiempo_ejecucion_MLPC,accuracy,"MLPC")

# Tiempos

In [36]:
print(f"Tiempo MLPC = {tiempo_ejecucion_MLPC:.2f} segundos")

Tiempo MLPC = 37.93 segundos
